In [ ]:
import os
input_name = "ky2.png"
base_name = os.path.splitext(input_name)[0]

building_map_name = f"{base_name}_building_map.png" #raster of buildings only
input_raster = building_map_name  # binary or probability raster
output_geojson = "f{base_name}.geojson"

In [ ]:
import cv2
import numpy as np

# Load image
img = cv2.imread(input_img)  # BGR format in OpenCV

# Split channels
b, g, r = cv2.split(img)

# Define threshold for red detection
threshold = 100  # tweak as needed

# Create binary mask: 1 if red is strong and green/blue are weak
building_map = np.where((r > threshold) & (g < threshold) & (b < threshold), 1, 0)

# Optional: save as image
cv2.imwrite(building_map_name, (building_map * 255).astype(np.uint8))


True

In [ ]:
import rasterio
from rasterio.features import shapes
import numpy as np
from skimage import measure, morphology
from shapely.geometry import shape, mapping
import fiona
import json

# --------------------------
# Parameters
# --------------------------
input_raster = building_map_name  # binary or probability raster
output_geojson = "ky2_buildings.geojson"
threshold = 0.5                     # if probability raster
simplify_tolerance = 2.0            # units in arbitrary coordinate system
min_area_pixels = 100                # remove small noise

# --------------------------
# Load raster
# --------------------------
with rasterio.open(input_raster) as src:
    raster = src.read(1)

# --------------------------
# Arbitrary transform
# --------------------------
# Let's assume top-left corner at (1000, 1000), each pixel = 1 unit
from rasterio.transform import from_origin
transform = from_origin(1000, 1000, 1, 1)

# --------------------------
# Threshold if raster is soft probability
# --------------------------
mask = raster > threshold

# --------------------------
# Remove small objects (noise)
# --------------------------
mask = morphology.remove_small_objects(mask.astype(bool), min_size=min_area_pixels)

# --------------------------
# Label connected components
# --------------------------
labels = measure.label(mask)

# --------------------------
# Polygonize each connected component
# --------------------------
polygons = []
for region in measure.regionprops(labels):
    coords = region.coords
    single_mask = np.zeros_like(mask, dtype=np.uint8)
    single_mask[tuple(coords.T)] = 1
    for geom, val in shapes(single_mask, mask=single_mask, transform=transform):
        if val == 1:
            poly = shape(geom)
            poly = poly.simplify(simplify_tolerance, preserve_topology=True)
            polygons.append(poly)

# --------------------------
# Remove empty geometries
# --------------------------
polygons = [poly for poly in polygons if poly.area > 0]

# --------------------------
# Save to GeoJSON
# --------------------------
geojson_features = []
for idx, poly in enumerate(polygons):
    geojson_features.append({
        "type": "Feature",
        "geometry": mapping(poly),
        "properties": {"id": idx}
    })

geojson_dict = {
    "type": "FeatureCollection",
    "features": geojson_features
}

with open(output_geojson, "w") as f:
    json.dump(geojson_dict, f)

print(f"Polygonized buildings saved to {output_geojson}")


Map geojson polygons to arbitrary coordinate space(because the raster picture is not geomspatially mapped) 

In [ ]:
import json
from shapely.geometry import shape, mapping
from shapely.ops import transform
import pyproj

# --------------------------
# Load existing GeoJSON
# --------------------------
input_geojson = "ky2_buildings.geojson"
output_geojson_4326 = "ky2_buildings_4326.geojson"

with open(input_geojson) as f:
    data = json.load(f)

# --------------------------
# Define projections
# --------------------------
# Treat the current arbitrary coordinates as EPSG:3857 (meters)
src_crs = pyproj.CRS("EPSG:3857")
dst_crs = pyproj.CRS("EPSG:4326")  # WGS84 lat/lon
project = pyproj.Transformer.from_crs(src_crs, dst_crs, always_xy=True).transform

# --------------------------
# Transform all polygons
# --------------------------
features_4326 = []
for feat in data["features"]:
    geom = shape(feat["geometry"])
    geom_4326 = transform(project, geom)
    features_4326.append({
        "type": "Feature",
        "geometry": mapping(geom_4326),
        "properties": feat["properties"]
    })

geojson_4326 = {
    "type": "FeatureCollection",
    "features": features_4326
}

# --------------------------
# Save transformed GeoJSON
# --------------------------
with open(output_geojson_4326, "w") as f:
    json.dump(geojson_4326, f)

print(f"Polygons converted to EPSG:4326 and saved to {output_geojson_4326}")
